## 날씨, 대기질, 코로나감염 현황을 알려주는 메신져봇 구현

In [1]:
import telegram

In [2]:
# 텔레그램 API 키 등록
with open('telegram_token.txt') as f:
    mytoken = f.read()
mytoken

'6540124535:AAFlmA7M3sEEaHtXK451bGTofe1dyEJ7miU'

In [3]:
# 텔레그램 봇 초기화
bot = telegram.Bot(token=mytoken)

In [4]:
from telegram.ext import Updater
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters

### 미세먼지 등급에 따른 이모지 출력 함수

In [5]:
def pm10grade(val):
    grade = '😊'
    try:
        val = int(val)
    except:
        return '🤬없다 이눔아!🤬'
    if val > 150: grade = '🥵'
    elif val > 80: grade = '🤔'
    elif val > 30: grade = '😗'
    return grade

In [6]:
def pm25grade(val):
    grade = '😊'
    try:
        val = int(val)
    except:
        return '🤬없다 이눔아!🤬'
    if val > 75: grade = '🥵'
    elif val > 35: grade = '🤔'
    elif val > 15: grade = '😗'
    return grade

### 대기정보 수집하고 조회하는 함수

In [7]:
import requests
import json

In [8]:
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty'
with open('datagokr_key.txt') as f:
    mykey = f.read()

In [18]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}
params = {'serviceKey':mykey, 'returnType':'json', 'ver':'1.0', 'numOfRows':1000, 'sidoName':'전국', 'pageNo':1}

In [10]:
# data.go.kr에서 미세먼지정보를 내려받아 json파일로 저장
def reload_air():
    res = requests.get(url, headers=headers, params=params)
    with open('./data/air.json', 'w', encoding='UTF-8') as f:
        f.writelines(res.text)

reload_air()

In [11]:
# 특정 지역의 미세먼지 정보를 조회해서 결과 반환
def fetch_air(region):
    with open('./data/air.json', encoding='UTF-8') as f:
        airdata = json.load(f)

    airinfo = ''
    for item in airdata['response']['body']['items']:
        if (region.split()[0] == item['sidoName']) and \
            (region.split()[1] == item['stationName']):
            #print(region, item['dataTime'], item['pm10Value'], item['pm25Value'])
            rate1 = pm10grade(item['pm10Value'])
            rate2 = pm25grade(item['pm25Value'])
            airinfo = f"{item['dataTime']} \n"\
                      f"{item['pm10Value']}㎍/㎥ {rate1} "\
                      f"{item['pm25Value']}㎍/㎥ {rate2} "
        
    return airinfo

#fetch_air('서울 관악구')

### 텔레그램 챗봇 응답처리 콜백함수

In [12]:
# 미세먼지 정보 출력
def air_message(update, context):
    airinfo = fetch_air(update.message.text)
    context.bot.send_message(update.effective_chat.id, airinfo)

In [13]:
# 미세먼지 정보 갱신 - /reload 입력시 호출
def reload_message(update, context):
    reload_air()
    context.bot.send_message(update.effective_chat.id, '💫대기정보 갱신됨!!💫')

### 텔레그램 챗봇 서버 초기화

In [14]:
print('텔레그램 RPA 서버 준비중...')
updater = Updater(mytoken, use_context=True)

텔레그램 RPA 서버 준비중...


#### 콜백함수 등록

In [15]:
air_handler = MessageHandler(Filters.text, air_message)
reload_handler = CommandHandler('reload', reload_message)

#### 핸들러 함수 등록

In [16]:
updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(air_handler)

#### 텔레그램 챗봇 서버 실행

In [17]:
print('텔레그램 RPA 서버 실행중...')
updater.start_polling(timeout=3, drop_pending_updates=True)
updater.idle()

텔레그램 RPA 서버 실행중...


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/handler.py", line 198, in handle_update
    return self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_19536/3690942836.py", line 4, in air_message
    context.bot.send_message(update.effective_chat.id, airinfo)
  File "/opt/conda/lib/python3.11/site-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/telegram/bot.py", line 513, in send_message
    return self._message(  # type: ignore[return-value]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/extbot.py"